In [54]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# Creating dummy data
np.random.seed(21)

In [3]:
num_samples = 1000

In [43]:
data = {
    'age': np.random.randint(18,65, size = num_samples),
    'bmi': np.random.uniform(18.5,40.0, size = num_samples),
    'cholesterol': np.random.randint(126,564, size = num_samples),
    'blood_pressure': np.random.randint(100,180, size = num_samples),
    'heart_rate': np.random.randint(60,100, size = num_samples),
}

In [44]:
patient_data = pd.DataFrame(data)

In [45]:
patient_data.head()

,age,bmi,cholesterol,blood_pressure,heart_rate
0,41,35.204935,234,131,92
1,62,35.869530,162,139,82
2,38,35.265045,467,176,90
3,44,22.216616,442,161,82
4,54,35.912471,163,147,93


In [46]:
patient_data['diabetes_risk'] = np.where(
    (patient_data['age'] > 45) & (patient_data['bmi'] > 30) |
    (patient_data['cholesterol'] > 240),
    1, #Risk
    0  # No Risk
)

In [47]:
patient_data.head()

,age,bmi,cholesterol,blood_pressure,heart_rate,diabetes_risk
0,41,35.204935,234,131,92,0
1,62,35.869530,162,139,82,1
2,38,35.265045,467,176,90,1
3,44,22.216616,442,161,82,1
4,54,35.912471,163,147,93,1


In [48]:
patient_data.to_csv('dummy_patient_data.csv', index = False)

Building Basic CDSS

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [66]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

class ClinicalDecisionSupportSystem:
    def __init__(self):
        self.knowledge_base = {
            'diabetes_risk': {
                'age_threshold': 55,
                'bmi_threshold': 30,
                'risk_factors': ['obesity', 'family_history']
            }
        }
        self.ml_model = None
        self.patient_data = None

    def load_patient_data(self, data_path):
        """Load patient data from CSV"""
        self.patient_data = pd.read_csv("/content/dummy_patient_data.csv")
        print("Patient data loaded successfully.")
        return self.patient_data

    def preprocess_data(self):
        """Preprocess and prepare data for modeling"""
        # Example preprocessing steps
        self.patient_data['age_risk'] = np.where(
            self.patient_data['age'] > self.knowledge_base['diabetes_risk']['age_threshold'],
            1, 0
        )
        self.patient_data['bmi_risk'] = np.where(
            self.patient_data['bmi'] > self.knowledge_base['diabetes_risk']['bmi_threshold'],
            1, 0
        )
        print("Preprocessing complete.")

    def train_ml_model(self, target_column):
        """Train machine learning model"""
        X = self.patient_data.drop(columns=['diabetes_risk'])
        y = self.patient_data['diabetes_risk']

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        self.ml_model = DecisionTreeClassifier()
        self.ml_model.fit(X_train, y_train)

        # Model evaluation
        y_pred = self.ml_model.predict(X_test)
        print("Model training complete.")
        print("Model Evaluation:")
        print(classification_report(y_test, y_pred))

    def preprocess_single_patient(self, patient_data):
        """Preprocess a single patient's data to align with training features."""
        patient_preprocessed = {
            'age': patient_data['age'], # Include 'age'
            'bmi': patient_data['bmi'], # Include 'bmi'
            'age_risk': 1 if patient_data['age'] > self.knowledge_base['diabetes_risk']['age_threshold'] else 0,
            'bmi_risk': 1 if patient_data['bmi'] > self.knowledge_base['diabetes_risk']['bmi_threshold'] else 0,
            'cholesterol': patient_data['cholesterol'],  # Changed 'cholestrol' to 'cholesterol'
            'blood_pressure': patient_data['blood_pressure'],
            'heart_rate': patient_data.get('heart_rate', 0) # Include 'heart_rate' and provide a default value if missing
        }

        # Match training feature order
        X_columns = list(self.patient_data.drop(columns=['diabetes_risk']).columns)
        return [patient_preprocessed[col] for col in X_columns]
    def generate_recommendation(self, patient_data):
        """Generate clinical recommendations"""
        # Preprocess the new patient data
        new_patient_features = self.preprocess_single_patient(patient_data)

        # Machine learning prediction
        ml_prediction = self.ml_model.predict([new_patient_features])

        # Combine rule-based and ML insights
        recommendation = {
            'ml_prediction': ml_prediction[0],
            'suggestions': []
        }

        if ml_prediction[0] == 1:
            recommendation['suggestions'].append("Consult a healthcare professional")
        else:
            recommendation['suggestions'].append("Maintain a healthy lifestyle")

        return recommendation

# Usage Example
def main():
    # Initialize the CDSS
    cdss = ClinicalDecisionSupportSystem()

    # Load and preprocess patient data
    data_path = '/content/dummy_patient_data.csv'  # Replace with your file path
    cdss.load_patient_data(data_path)
    cdss.preprocess_data()

    # Train the ML model
    cdss.train_ml_model(target_column='diabetes_risk')

    # Example new patient data
    new_patient = {
        'age': 55,
        'bmi': 30,
        'cholesterol': 160,
        'blood_pressure': 240 # SBP taken
    }

    # Generate recommendations for the new patient
    recommendation = cdss.generate_recommendation(new_patient)
    print("Recommendation:", recommendation)

if __name__ == "__main__":
    main()


Patient data loaded successfully.
Preprocessing complete.
Model training complete.
Model Evaluation:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        45
           1       1.00      1.00      1.00       155

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Recommendation: {'ml_prediction': 0, 'suggestions': ['Maintain a healthy lifestyle']}


To save the model for future use

In [63]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import joblib  # Import the joblib library for saving and loading models

class ClinicalDecisionSupportSystem:
    def __init__(self):
        self.ml_model = None

    def train_ml_model(self, data, target_column):
        """Train the machine learning model"""
        X = data.drop(columns=[target_column])
        y = data[target_column]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.ml_model = DecisionTreeClassifier()
        self.ml_model.fit(X_train, y_train)

        accuracy = self.ml_model.score(X_test, y_test)
        print(f"Model Accuracy: {accuracy * 100:.2f}%")

        # Save the model to a file
        joblib.dump(self.ml_model, "clinical_decision_model.pkl")
        print("Model saved as 'clinical_decision_model.pkl'")

# Example usage:
def main():
    # Example dummy dataset
    import pandas as pd
    data = pd.DataFrame({
        'age': [25, 45, 50, 30, 60],
        'bmi': [22, 30, 35, 25, 40],
        'cholesterol': [180, 220, 200, 190, 230],
        'diabetes_risk': [0, 1, 1, 0, 1]
    })

    cdss = ClinicalDecisionSupportSystem()
    cdss.train_ml_model(data, target_column="diabetes_risk")

if __name__ == "__main__":
    main()


Model Accuracy: 100.00%
Model saved as 'clinical_decision_model.pkl'


To load the model in future

In [65]:
import joblib

class ClinicalDecisionSupportSystem:
    def __init__(self):
        self.ml_model = None

    def load_model(self, file_path):
        """Load a saved machine learning model"""
        self.ml_model = joblib.load(file_path)
        print(f"Model loaded from '{file_path}'")

    def generate_recommendation(self, patient_data):
        """Generate a recommendation using the loaded model"""
        if not self.ml_model:
            raise ValueError("Model not loaded. Please load a model first.")

        # Prepare the patient data in the correct format for prediction
        patient_features = [list(patient_data.values())]
        prediction = self.ml_model.predict(patient_features)

        recommendation = {
            'ml_prediction': prediction[0],
            'suggestions': []
        }

        if prediction[0] == 1:
            recommendation['suggestions'].append("Consult a healthcare professional")
        else:
            recommendation['suggestions'].append("Maintain a healthy lifestyle")

        return recommendation

# Example usage:
def main():
    # New patient data
    new_patient = {
        'age': 25,
        'bmi': 32,
        'cholesterol': 220
    }

    cdss = ClinicalDecisionSupportSystem()
    cdss.load_model("clinical_decision_model.pkl")

    recommendation = cdss.generate_recommendation(new_patient)
    print(recommendation)

if __name__ == "__main__":
    main()


Model loaded from 'clinical_decision_model.pkl'
{'ml_prediction': 0, 'suggestions': ['Maintain a healthy lifestyle']}
